In [ ]:
!rm -rf /kaggle/working/multimodal-eq-sizing
!git clone https://github.com/brianrp09232000/multimodal-eq-sizing.git /kaggle/working/multimodal-eq-sizing
!pip install -r /kaggle/working/multimodal-eq-sizing/requirements.txt

In [ ]:
import sys
import pathlib
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
np.seterr(invalid="ignore")

In [ ]:
repo_root = pathlib.Path("/kaggle/working/multimodal-eq-sizing")
sys.path.append(str(repo_root))

In [ ]:
from src.data.loaders import (
    get_tickers_history,
    get_return_data,
    get_excess_return,
    get_vix_data,
    get_spread_z,
    get_sector_map,
    get_adv_dollar
)

# Find Tickers with the Most Headlines

In [ ]:
from src.data.universe import tickers_with_most_headlines

In [ ]:
news_df = pd.read_csv("/kaggle/input/financial-news/combined.csv") 

In [ ]:
def count_headlines_per_ticker(news_df, start=None, end=None):
    """Counts the number of headlines for each ticker symbol 
    Input: news_df pandas dataframe with ticker column for ticker symbols
    Output: pandas dataframe containing two columns: ticker names and the 
                number of headlines for the ticker"""
    
    #check columns in dataframe
    columns = list(news_df.columns)
    if (('date' not in columns) and ('Date' not in columns)) or (('ticker' not in columns) and ('Stock_symbol' not in columns)):
        print('input dataframe does not have both ticker and date columns')
        return pd.DataFrame()
    
    #find column names
    date_col = 'date' if 'date' in columns else 'Date'
    ticker_col = 'ticker' if 'ticker' in columns else 'Stock_symbol'

    #filter dates
    if start is not None: 
        start_filter = news_df[date_col] >= str(start)
        news_df = news_df[start_filter]
    if end is not None: 
        end_filter = news_df[date_col] <= str(end)
        news_df = news_df[end_filter]
    
    # Count occurrences in a specific column
    headline_counts = news_df[ticker_col].value_counts()
    df = headline_counts.to_frame(name='count')
    df['ticker'] = list(df.index)
    df = df.reset_index(drop=True)
    
    return df[['ticker','count']]



def tickers_with_most_headlines(news_df, start=None, end=None, n=200):
    """Finds the tickers with the most headlines 
    Input: news_df pandas dataframe with ticker column for ticker symbols
            optional: n interger, number of top tickers to return
    Output: list containing the number of headlines per ticker
                for the tickers with the most headlines"""
    
    #count headlines for each ticker
    df = count_headlines_per_ticker(news_df, start, end)

    #limit dataframe to n tickers
    df = df.sort_values(['count'], ascending=False)
    df = df[:n]
    df.reset_index(drop=True, inplace=True)
    
    return df

In [ ]:
start = pd.Timestamp('2010-01-04 05:00:00+0000', tz='UTC')
end   = pd.Timestamp('2018-12-28 05:00:00+0000', tz='UTC')
tickers = tickers_with_most_headlines(news_df, str(start), str(end), 300)

# Get Price Data

In [ ]:
#get yfinance ticker history for all tickers in tickers df
#yfinance will produce the "possibly delisted" message for tickers without information
df = get_tickers_history(list(tickers['ticker']), start=start, end=end)

In [ ]:
#limit df to only 200 tickers and tickers with data
keep_tickers = list(df['ticker'].drop_duplicates()[:200])
df = df[df['ticker'].isin(keep_tickers)]
tickers = tickers[tickers['ticker'].isin(keep_tickers)]

In [ ]:
def get_date_range(df: pd.DataFrame) -> tuple:
    grouped_by_date = df.groupby(["ticker"]).agg(['min', 'max', 'count'])["Date"]
    start = grouped_by_date["min"].min()
    end = grouped_by_date["max"].max()
    return start, end

In [ ]:
#df = get_return_data("/kaggle/input/news-trading/return_data.csv")
#start, end = get_date_range(df)

# Add excess return

In [ ]:
def add_excess_return(df, start, end):
    excess_return_df = get_excess_return(df, start, end)
    df = df.merge(excess_return_df, on=["ticker", "Date"], how="left")
    return df

In [ ]:
df = add_excess_return(df, start, end)

# Add market regime VIX z-score

In [ ]:
def add_vix_z(df, start, end):
    vix_z_df = get_vix_data(start, end)
    format_str = "%Y-%m-%d"
    vix_z_df["Date"] = vix_z_df["Date"].dt.strftime(format_str)
    df["Date"] = df["Date"].dt.strftime(format_str) 
    df = df.merge(vix_z_df, on=["Date"], how="left")
    df['Date'] = pd.to_datetime(df['Date'], utc=True)
    return df

In [ ]:
df = add_vix_z(df, start, end)

# Add spread z-score

In [ ]:
def add_spread_z(existing_df: pd.DataFrame, buffer_days=380) -> pd.DataFrame:
    """
    Use existing OHLCV df, pull buffered history, compute young-safe spread_z on the combined
    Then merge back only the target window rows to prevent nulls.
    """
    df = existing_df.copy()
    start, end = df["Date"].min(), df["Date"].max()

    tickers = sorted(df['ticker'].unique())
    fetch_start = start - timedelta(days=buffer_days)
    fetch_end   = end

    # You already have get_tickers_history(tickers, start, end)
    hist = get_tickers_history(tickers, fetch_start, fetch_end)
    hist["Date"] = pd.to_datetime(hist["Date"], utc=True)

    # Combine buffer + existing; keep existing rows on overlap
    combined = pd.concat([hist, df], ignore_index=True)
    combined = combined.sort_values(['ticker', "Date"])
    combined = combined.drop_duplicates(subset=['ticker', "Date"], keep="last")

    # Compute young-safe spread_z on the full combined range
    combined = get_spread_z(combined)

    # Merge only computed columns back to target window
    cols_to_merge = ['ticker', 'Date', "spread_z"]
    out = df.merge(combined[cols_to_merge], on=['ticker', 'Date'], how="left")

    # Final minimal, causal clean-up to guarantee NON-NULL spread_z in target window:
    # 1) per-ticker forward-fill (past only), 2) same-day cross-section median, 3) final 0
    out["spread_z"] = (
        out.groupby('ticker')["spread_z"].ffill()
           .fillna(out.groupby('Date')["spread_z"].transform("median"))
           .fillna(0.0)
    ).clip(-3, 3)

    return out

In [ ]:
df = add_spread_z(df)

# Add sector

In [ ]:
def add_sector(df):
    tickers = df["ticker"].unique()
    sector_map = get_sector_map(tickers)
    df = df.join(sector_map, on="ticker")
    return df

In [ ]:
df = add_sector(df)

# Add dollar-volume ADV 

In [ ]:
def add_adv_dollar(df):
    adv_df = get_adv_dollar(df)
    
    df = df.merge(
        adv_df,
        on=["Date", "ticker"],
        how="left",
    )
    return df

In [ ]:
df = add_adv_dollar(df)

# Add Next Day Excess

In [ ]:
df["next_day_excess_return"] = df.groupby('ticker')['excess_return'].shift(-1)

In [ ]:
df_org = df.copy()

In [ ]:
df

# Add News Flag

In [ ]:
from src.data.features.news_features import count_headlines_all_days


def add_news_flag(news_df, price_df, start=None, end=None):
    """adds a new news flag column: 0=no news, 1=news
    input: news_df with 'date', 'ticker', and other columns
            price_df with 'Date', 'ticker', and other columns
            optional start and end Timestamps
    output: dataframe df
    """

    #count headlines per ticker per day
    news_count = count_headlines_all_days(news_df)

    #filter count_df by date
    news_count = news_count.T
    news_count['date'] = pd.to_datetime(list(news_count.index), utc=True)
    if start is not None: news_count = news_count[news_count['date'] >= start]
    if end is not None: news_count = news_count[news_count['date'] <= end]
    
    #convert news_count df to different format
    news_cols = list(news_count.columns)
    news_count = news_count.melt(id_vars=['date'], value_vars=news_cols, 
                  var_name='ticker', value_name='news flag')

    # change count to flag: 0=no news, 1=news
    news_count['news flag'] = [flag if flag < 2 else 1 for flag in news_count['news flag']]
    news_count['date'] = pd.to_datetime(news_count['date'], utc=True)
    news_count.sort_values(['date','ticker'], inplace=True)
    
    #add news flag: 0=no news, 1=news
    price_df = pd.merge(price_df, news_count, left_on=['Date','ticker'], 
              right_on=['date','ticker'])

    return price_df

In [ ]:
df = add_news_flag(news_df, df, start=start, end=end)

# Split Data

In [ ]:
df['split'] = 'train'
df.loc[df['Date'] >="2015-01-01", "split"] = "val"
df.loc[df['Date'] >= "2017-01-01", "split"] = 'test'

# Final Complete Dataset

In [ ]:
df.to_csv('final_dataset.csv', index=False)

In [ ]:
tickers.to_csv('top_tickers.csv', index=False)

In [ ]:
get_return_data("/kaggle/working/final_dataset.csv")

In [ ]:
!rm -rf /kaggle/working/multimodal-eq-sizing